In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
def add_intercept_1d(x):
    return np.c_[np.ones((x.shape[0], 1)), x]

In [3]:
def ridge_gradient_descent(X, y, alpha, num_iters, lam):
    n, d = X.shape
    theta = np.zeros((d, 1))
    for _ in range(num_iters):
        error = (X @ theta) - y
        grad = (2 / n) * (X.T @ error)

        reg = (2 * lam / n) * theta
        reg[0] = 0

        theta = theta - alpha * (grad + reg)
    return theta

In [4]:
def predict(X, theta):
    return X @ theta

In [5]:
np.random.seed(42)

N = 1000
X = np.random.uniform(-2, 2, size=(N, 1))

e = np.random.normal(loc=0.0, scale=np.sqrt(2), size=(N, 1))

Y = 1 + 2 * X + e

X_b = add_intercept_1d(X)

In [6]:
alpha = 0.1
num_iters = 5000

theta_lin = ridge_gradient_descent(X_b, Y, alpha=alpha, num_iters=num_iters, lam=0)

Y_pred_lin = predict(X_b, theta_lin)
mse_lin = mean_squared_error(Y, Y_pred_lin)
r2_lin = r2_score(Y, Y_pred_lin)

print("Linear Regression (GD) results")
print("theta0 (intercept):", theta_lin[0, 0])
print("theta1 (slope):    ", theta_lin[1, 0])
print("MSE:", mse_lin)
print("R^2:", r2_lin)
print("-" * 60)

Linear Regression (GD) results
theta0 (intercept): 1.1377269735725186
theta1 (slope):     1.94527518078825
MSE: 1.9499357635589158
R^2: 0.725823805647487
------------------------------------------------------------


In [7]:
alpha = 0.01
num_iters = 5000

lambdas = [1, 10, 100, 1000, 10000]
rows = []

for lam in lambdas:
    theta_r = ridge_gradient_descent(X_b, Y, alpha=alpha, num_iters=num_iters, lam=lam)
    Y_pred_r = predict(X_b, theta_r)

    if (not np.isfinite(theta_r).all()) or (not np.isfinite(Y_pred_r).all()):
        rows.append({
            "lambda": lam,
            "theta0": np.nan,
            "theta1 (slope)": np.nan,
            "MSE": np.nan,
            "R2": np.nan,
        })
        continue

    rows.append({
        "lambda": lam,
        "theta0": theta_r[0, 0],
        "theta1 (slope)": theta_r[1, 0],
        "MSE": mean_squared_error(Y, Y_pred_r),
        "R2": r2_score(Y, Y_pred_r),
    })

results_df = pd.DataFrame(rows)
print(results_df)

   lambda    theta0  theta1 (slope)       MSE        R2
0       1  1.137671        1.943850  1.949939  0.725823
1      10  1.137175        1.931119  1.950209  0.725785
2     100  1.132549        1.812414  1.974016  0.722438
3    1000  1.105658        1.122450  2.873516  0.595961
4   10000  1.071013        0.233509  5.947067  0.163796
